In [ ]:
import pandas as pd
from utils import *
import itertools

## Affected Importers Label

A country is labelled as affected importer if:
 * One of its exporter partners has "lost" its condition as exporter
 * The importer either reduces its imports for the commodity by an amount of 20%; OR
 * The importer pays a higher price than the previous year, in excess of the price change for the commodity

Retrieve BACI data to compute average prices

In [ ]:
avg_prices = pd.DataFrame()
avg_prices_per_importer = pd.DataFrame()
atlas_countries = pd.concat([pd.read_stata("../../data/2. Atlas//hs12_country_country_product_year_4_2012_2016.dta"),
                pd.read_stata("../../data/2. Atlas/hs12_country_country_product_year_4_2017_2021.dta"),
                pd.read_stata("../../data/2. Atlas/hs12_country_country_product_year_4_2022.dta")]).country_id.unique()
all_countries = set(atlas_countries)

for year in range(2012, 2023):
    baci = pd.read_csv(f"../../data/3. BACI/BACI_HS12_Y{year}_V202501.csv", dtype={"product_id": str})
    baci["product_code"] = baci.product_id.str[:2] ## Reduce product code to 2 digits
    baci.dropna(inplace=True) # Some entries have NANs for qty. We'll dismiss this faulty entries.
    # Calculate the average price each importer was paying for each product (across all partners)
    avg_prices_per_importer_year = baci.groupby(["importer", "product_code"])[["export_value", "qty"]].sum().reset_index()
    avg_prices_per_importer_year["price"] = avg_prices_per_importer_year.export_value / avg_prices_per_importer_year.qty
    avg_prices_per_importer_year.loc[:, ["year"]] = year
    avg_prices_per_importer = pd.concat([avg_prices_per_importer, avg_prices_per_importer_year])
    # Calculate average price per commodity-year
    avg_prices_year = baci.groupby("product_code")[["export_value", "qty"]].sum().reset_index()
    avg_prices_year["price"] = avg_prices_year.export_value / avg_prices_year.qty
    avg_prices_year["year"] = year
    avg_prices = pd.concat([avg_prices, avg_prices_year])
    # Update all seen countries
    all_countries.update(baci.exporter)
    all_countries.update(baci.importer)
    del baci # to recover some memory

Get the report of potentially affected importers by 'lost exporters'

In [ ]:
# Initial report of potentially affected importers from lost exporters
drop_exporters_impact = pd.read_csv("../../data/df_impact_drop_exporters.csv")
drop_exporters_impact.rename(columns={"pid": "product_code"}, inplace=True)
drop_exporters_impact.head(3)

In [ ]:
# Calculate drop in imports from potentially affected importers
drop_exporters_impact["importer_drop"] = (drop_exporters_impact["year_to_value"] / drop_exporters_impact["year_from_value"]) - 1
drop_exporters_impact.head(3)

We need to find the products that 'affected importer' was importing from 'lost exporters' at 'year from', then add the avg. price of the commodity, then do the same for 'year to', and calculate the ratios.

In [ ]:
# Get Avg. Price Product (APP) for year from
drop_exporters_impact = drop_exporters_impact.merge(avg_prices[["product_code", "year", "price"]], left_on=["product_code", "year_from"], \
                                                    right_on=["product_code", "year"], how="left")
drop_exporters_impact.drop("year", axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "APP_year_from"}, inplace=True)
# Get Avg. Price Product (APP) for year to
drop_exporters_impact = drop_exporters_impact.merge(avg_prices[["product_code", "year", "price"]], left_on=["product_code", "year_to"], \
                                                    right_on=["product_code", "year"], how="left")
drop_exporters_impact.drop("year", axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "APP_year_to"}, inplace=True)
# APP Change
drop_exporters_impact.loc[:, ["APP_Change"]] = (drop_exporters_impact.APP_year_to / drop_exporters_impact.APP_year_from) - 1

In [ ]:
drop_exporters_impact.head(3)

We need to find the products that 'affected importer' was importing from 'lost exporters' at 'year from', then add the avg. price paid by importer and avg. price of the commodity, then do the same for 'year to', and calculate the ratios.

In [ ]:
# Get Avg. Price Product (APP) for year from
drop_exporters_impact = drop_exporters_impact.merge(avg_prices_per_importer[["importer", "product_code", "year", "price"]], \
                                                    left_on=["affected_importer", "product_code", "year_from"], \
                                                    right_on=["importer", "product_code", "year"], how="left")
drop_exporters_impact.drop(["importer", "year"], axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "CPP_year_from"}, inplace=True)
# Get Avg. Price Product (APP) for year to
drop_exporters_impact = drop_exporters_impact.merge(avg_prices_per_importer[["importer", "product_code", "year", "price"]], \
                                                    left_on=["affected_importer", "product_code", "year_to"], \
                                                    right_on=["importer", "product_code", "year"], how="left")
drop_exporters_impact.drop(["importer", "year"], axis=1, inplace=True)
drop_exporters_impact.rename(columns={"price": "CPP_year_to"}, inplace=True)
# APP Change
drop_exporters_impact.loc[:, ["expected_change"]] = drop_exporters_impact.CPP_year_from * (1+drop_exporters_impact.APP_Change)

In [ ]:
drop_exporters_impact["net_change"] = (drop_exporters_impact.CPP_year_to / drop_exporters_impact.expected_change) - 1

Apply conditions for affected label

In [ ]:
affected_importers = drop_exporters_impact[(drop_exporters_impact.importer_drop <= -0.2) | (drop_exporters_impact.net_change >= 0.2)]

Make sure all labels are present

In [ ]:
reporters = pd.read_csv("../../data/1. UN Comtrade/reporters.csv")
reporters.rename(columns={"reporterCode": "country_id"}, inplace=True)
all_countries.update(reporters.country_id) # Add countries from reporters (UN Comtrade)

# Generate all combinations
all_combinations = pd.DataFrame(itertools.product(all_countries, list(range(2012,2022)), \
                                                  list([f'{x:02d}' for x in range(1, 99) if x not in [77, 98]])), columns=['country_id', 'year', 'product_code'])

In [ ]:
affected_importers.head()

In [ ]:
affected_importers.info()

In [ ]:
affected_importers.fillna(0, inplace=True)

In [ ]:
affected_importers["label"] = "affected_importer"
affected_importers.rename(columns={"affected_importer": "country_id", "year_from": "year"}, inplace=True)
affected_importers = all_combinations.merge(affected_importers[["country_id", "year", "product_code", "label"]], on=["year", "product_code", "country_id"], how="left")
affected_importers.fillna("not_affected", inplace=True)

In [ ]:
affected_importers.head(3)

In [ ]:
affected_importers.groupby(["year", "product_code", "label"]).size()

In [ ]:
affected_importers.sort_values(["year", "product_code", "country_id"])[["year", "product_code", "country_id", "label"]].to_csv("../../data/labels-affected_importers.csv", index=False)